In [3]:
import pandas as pd
import numpy as np
import pickle


%matplotlib inline
#settings
pd.set_option('display.max_colwidth', -1)
np.set_printoptions(threshold=np.inf)
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"
# init_notebook_mode(connected=True)
%config InlineBackend.figure_format = 'svg'
%matplotlib inline 
seed = 123

In [5]:
with open('../data/processed/df_for_model_2.pkl', 'rb') as read_file:
    df_for_model_2 = pickle.load(read_file)   
with open('../data/processed/df_for_model_3.pkl', 'rb') as read_file:
    df_for_model_3 = pickle.load(read_file)   
# with open('../data/processed/master_df.pkl', 'rb') as read_file:
#     master_df = pickle.load(read_file)   

In [51]:
master_df.head(1)

,age,age_group,"age_group_[10, 30)","age_group_[30, 40)","age_group_[40, 50)","age_group_[50, 60)","age_group_[60, 96)",balance,balance_category,"balance_category_[-8019, 0)","balance_category_[0, 500)","balance_category_[3000, 102128)","balance_category_[500, 3000)",campaign,cluster_predicted,contact,contact_label_encoded,contact_label_encoded_0,contact_label_encoded_1,contact_label_encoded_2,current_engagement_category,"current_engagement_category_[1, 6)","current_engagement_category_[10, 64)","current_engagement_category_[6, 10)",day,default,default_label_encoded,default_label_encoded_0,default_label_encoded_1,duration,duration_category,"duration_category_[0, 60)","duration_category_[180, 300)","duration_category_[300, 4919)","duration_category_[60, 180)",education,education_label_encoded,education_label_encoded_0,education_label_encoded_1,education_label_encoded_2,education_label_encoded_3,housing,housing_label_encoded,housing_label_encoded_0,housing_label_encoded_1,ID,is_second_half_of_month,is_second_half_of_month_0,is_second_half_of_month_1,job,job_label_encoded,job_label_encoded_0,job_label_encoded_1,job_label_encoded_10,job_label_encoded_11,job_label_encoded_2,job_label_encoded_3,job_label_encoded_4,job_label_encoded_5,job_label_encoded_6,job_label_encoded_7,job_label_encoded_8,job_label_encoded_9,level_0,loan,loan_label_encoded,loan_label_encoded_0,loan_label_encoded_1,marital,marital_label_encoded,marital_label_encoded_0,marital_label_encoded_1,marital_label_encoded_2,month,month_label_encoded,month_label_encoded_1,month_label_encoded_10,month_label_encoded_11,month_label_encoded_12,month_label_encoded_2,month_label_encoded_3,month_label_encoded_4,month_label_encoded_5,month_label_encoded_6,month_label_encoded_7,month_label_encoded_8,month_label_encoded_9,new_customers,new_customers_0,new_customers_1,pdays,pdays_category,"pdays_category_[-1, 0)","pdays_category_[0, 180)","pdays_category_[180, 872)",poutcome,poutcome_label_encoded,poutcome_label_encoded_0,poutcome_label_encoded_1,poutcome_label_encoded_2,poutcome_label_encoded_3,prev_engagement_category,"prev_engagement_category_[0, 1)","prev_engagement_category_[1, 6)","prev_engagement_category_[10, 276)","prev_engagement_category_[6, 10)",previous,total_duration,total_duration_category,"total_duration_category_[0, 180)","total_duration_category_[180, 600)","total_duration_category_[600, 60171)",y,y_label_encoded,age_group_label_encoded,balance_category_label_encoded,current_engagement_category_label_encoded,duration_category_label_encoded,pdays_category_label_encoded,prev_engagement_category_label_encoded,total_duration_category_label_encoded,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
0,39,"[30, 40)",0,1,0,0,0,816,"[500, 3000)",0,0,0,1,3,2,cellular,0,1,0,0,"[1, 6)",1,0,0,27,no,0,1,0,37,"[0, 60)",1,0,0,0,tertiary,2,0,0,1,0,yes,1,0,1,23354,1,0,1,management,4,0,0,0,0,0,0,1,0,0,0,0,0,0.0,no,0,1,0,single,2,0,0,1,aug,8,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,-1,"[-1, 0)",1,0,0,unknown,3,0,0,0,1,"[0, 1)",1,0,0,0,0,111,"[0, 180)",1,0,0,no,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [35]:
df_for_model_2.columns.tolist()[:21]

['ID',
 'cluster_predicted',
 'age_group',
 'balance_category',
 'contact_label_encoded',
 'current_engagement_bins',
 'default_label_encoded',
 'duration_bins',
 'education_label_encoded',
 'housing_label_encoded',
 'job_label_encoded',
 'marital_label_encoded',
 'loan_label_encoded',
 'month_label_encoded',
 'poutcome_label_encoded',
 'pdays_half_yearly_bins',
 'prev_engagement_category',
 'is_second_half_of_month',
 'day',
 'new_customers',
 'y_label_encoded']

In [6]:
# non_ohe_variables = master_df.select_dtypes(include = ['category', 'int64'] ).columns.tolist()
# non_ohe_variables.remove("ID")
# # non_ohe_variables_df = master_df[non_ohe_variables]
# non_cat_features = ['age', 'balance', 'campaign', 'duration', 'day', 'cluster_predicted', 'previous', 'pdays', 'total_duration']

iv_features = df_for_model_2.columns.tolist()[:21]
iv_features.remove('ID')
len(iv_features)
iv_df = df_for_model_2[iv_features]


20

In [7]:
def calculate_woe_iv(dataset, feature, target):
    iv_list = []
    for i in range(dataset[feature].nunique()):
        val = list(dataset[feature].unique())[i]
        iv_list.append({
            'Value': val,
            'All': dataset[dataset[feature] == val].count()[feature],
            'No': dataset[(dataset[feature] == val) & (dataset[target] == 0)].count()[feature],
            'Yes': dataset[(dataset[feature] == val) & (dataset[target] == 1)].count()[feature]
        })
        
    dset = pd.DataFrame(iv_list)
    dset['Distr_Yes'] = dset['Yes'] / dset['Yes'].sum()
    dset['Distr_No'] = dset['No'] / dset['No'].sum()
    dset['WoE'] = np.log(dset['Distr_Yes'] / dset['Distr_No'])
    dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    dset['IV'] = (dset['Distr_Yes'] - dset['Distr_No']) * dset['WoE']
    iv = dset['IV'].sum()
    
    dset = dset.sort_values(by='WoE')
    
    return dset, iv

In [8]:
iv_values = []
for col in iv_df.columns:
    if col == 'y_label_encoded': continue
    else:
        print('WoE and IV for column: {}'.format(col))
        df, iv = calculate_woe_iv(iv_df, col, 'y_label_encoded')
        iv_values.append(iv.round(2))
        print(df)
        print('IV score: {:.2f}'.format(iv))
        print('\n')
        

WoE and IV for column: cluster_predicted
   Value    All     No   Yes  Distr_Yes  Distr_No       WoE        IV
4  1      10803  10293  510   0.096427   0.257828 -0.983510  0.158740
3  3      5356   4852   504   0.095292   0.121537 -0.243272  0.006385
1  0      12544  10999  1545  0.292116   0.275512  0.058518  0.000972
0  2      8931   7806   1125  0.212706   0.195531  0.084189  0.001446
2  4      7577   5972   1605  0.303460   0.149592  0.707340  0.108837
IV score: 0.28


WoE and IV for column: age_group
      Value    All     No   Yes  Distr_Yes  Distr_No       WoE        IV
1  [40, 50)  11655  10592  1063  0.200983   0.265317 -0.277706  0.017866
3  [50, 60)  8410   7625   785   0.148421   0.190997 -0.252206  0.010738
0  [30, 40)  18089  16176  1913  0.361694   0.405190 -0.113558  0.004939
2  [10, 30)  5273   4345   928   0.175458   0.108837  0.477549  0.031815
4  [60, 96)  1784   1184   600   0.113443   0.029658  1.341574  0.112404
IV score: 0.18


WoE and IV for column: balance_cat

In [9]:
list_of_iv_values = sorted(list(zip(iv_features, iv_values)),key=lambda x: x[1], reverse = True)
iv_features_df = pd.DataFrame(list_of_iv_values, columns =['Feature', 'IV_value']) 
iv_features_df
iv_features_df.to_csv('../data/processed/iv_features_df.csv')

,Feature,IV_value
0,duration_bins,1.56
1,poutcome_label_encoded,0.51
2,month_label_encoded,0.44
3,contact_label_encoded,0.30
4,cluster_predicted,0.28
5,pdays_half_yearly_bins,0.24
6,prev_engagement_category,0.22
7,new_customers,0.22
8,housing_label_encoded,0.19
9,age_group,0.18


In [3]:
df_for_model_2.head(2)

,ID,cluster_predicted,age_group,balance_category,contact_label_encoded,current_engagement_bins,default_label_encoded,duration_bins,education_label_encoded,housing_label_encoded,job_label_encoded,marital_label_encoded,loan_label_encoded,month_label_encoded,poutcome_label_encoded,pdays_half_yearly_bins,prev_engagement_category,is_second_half_of_month,day,new_customers,y_label_encoded,cluster_predicted_0,cluster_predicted_1,cluster_predicted_2,cluster_predicted_3,cluster_predicted_4,"age_group_[10, 30)","age_group_[30, 40)","age_group_[40, 50)","age_group_[50, 60)","age_group_[60, 96)","balance_category_[-8019, 0)","balance_category_[0, 500)","balance_category_[500, 3000)","balance_category_[3000, 102128)",contact_label_encoded_0,contact_label_encoded_1,contact_label_encoded_2,"current_engagement_bins_[1, 2)","current_engagement_bins_[2, 6)","current_engagement_bins_[6, 11)","current_engagement_bins_[11, 64)",default_label_encoded_0,default_label_encoded_1,"duration_bins_[0, 60)","duration_bins_[60, 120)","duration_bins_[120, 300)","duration_bins_[300, 600)","duration_bins_[600, 900)","duration_bins_[900, 4919)",education_label_encoded_0,education_label_encoded_1,education_label_encoded_2,education_label_encoded_3,housing_label_encoded_0,housing_label_encoded_1,job_label_encoded_0,job_label_encoded_1,job_label_encoded_2,job_label_encoded_3,job_label_encoded_4,job_label_encoded_5,job_label_encoded_6,job_label_encoded_7,job_label_encoded_8,job_label_encoded_9,job_label_encoded_10,job_label_encoded_11,marital_label_encoded_0,marital_label_encoded_1,marital_label_encoded_2,loan_label_encoded_0,loan_label_encoded_1,month_label_encoded_1,month_label_encoded_2,month_label_encoded_3,month_label_encoded_4,month_label_encoded_5,month_label_encoded_6,month_label_encoded_7,month_label_encoded_8,month_label_encoded_9,month_label_encoded_10,month_label_encoded_11,month_label_encoded_12,poutcome_label_encoded_0,poutcome_label_encoded_1,poutcome_label_encoded_2,poutcome_label_encoded_3,"pdays_half_yearly_bins_[-1, 0)","pdays_half_yearly_bins_[0, 180)","pdays_half_yearly_bins_[180, 872)","prev_engagement_category_[0, 1)","prev_engagement_category_[1, 6)","prev_engagement_category_[6, 10)","prev_engagement_category_[10, 276)",is_second_half_of_month_0,is_second_half_of_month_1,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,new_customers_0,new_customers_1,age_group_label_encoded,balance_category_label_encoded,current_engagement_bins_label_encoded,duration_bins_label_encoded,pdays_half_yearly_bins_label_encoded,prev_engagement_category_label_encoded
0,23354,2,"[30, 40)","[500, 3000)",0,"[2, 6)",0,"[0, 60)",2,1,4,2,0,8,3,"[-1, 0)","[0, 1)",1,27,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,3,2,0,0,0
1,18555,0,"[40, 50)","[0, 500)",0,"[11, 64)",0,"[60, 120)",1,1,7,1,0,7,3,"[-1, 0)","[0, 1)",1,31,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,2,1,1,3,0,0


In [11]:
#Removed default_label_encoded, is_second_half_of_month   
# df_variables_1 = df_for_model_2[['ID','age_group_label_encoded', 'balance_category_label_encoded', 'cluster_predicted', 'contact_label_encoded', 'current_engagement_bins_label_encoded',
#                'day', 'duration_bins_label_encoded', 'education_label_encoded', 'housing_label_encoded', 'job_label_encoded', 'loan_label_encoded', 'marital_label_encoded', 'month_label_encoded',
#                'new_customers', 'pdays_half_yearly_bins_label_encoded', 'poutcome_label_encoded', 'prev_engagement_category_label_encoded', 'y_label_encoded']] 
train_val_df, test_set_df = train_test_split(df_for_model_2, test_size=0.2,random_state=seed)


In [14]:
# ('new_customers')['balance_category']
train_val_df.pivot_table(index = 'new_customers', columns = ['balance_category', 'y_label_encoded'], values = 'ID', aggfunc = len)


balance_category [-8019, 0)      [0, 500)       [500, 3000)        \
y_label_encoded           0    1        0     1           0     1   
new_customers                                                       
0                 380        31   2058     556   2016        683    
1                 2516       142  12187    1041  8989        1103   

balance_category [3000, 102128)       
y_label_encoded               0    1  
new_customers                         
0                 683            279  
1                 3075           429

In [17]:
new_customers_by_duration_and_target = train_val_df.pivot_table(index = 'new_customers', columns = ['duration_bins', 'y_label_encoded'], values = 'ID', aggfunc = len,margins=True)
new_customers_by_duration_and_target_percen = (new_customers_by_duration_and_target.div(new_customers_by_duration_and_target.iloc[:,-1], axis=0 )*100).round(2)
new_customers_by_duration_and_target_percen


duration_bins   [0, 60)       [60, 120)       [120, 300)        [300, 600)  \
y_label_encoded       0     1         0     1          0      1          0   
new_customers                                                                
0                8.62    0.01  17.72     0.90  34.19      10.08  13.04       
1                10.68   0.02  20.50     0.31  39.09      2.20   15.89       
All              10.30   0.02  19.99     0.41  38.19      3.66   15.36       

duration_bins         [600, 900)       [900, 4919)          All  
y_label_encoded     1          0     1           0     1         
new_customers                                                    
0                7.69  2.23       2.92  1.03        1.57  100.0  
1                2.68  3.30       2.06  1.33        1.95  100.0  
All              3.60  3.10       2.22  1.28        1.88  100.0

## 2.Post k-means clustering IV 

In [58]:
set(df_for_model_3.columns.tolist()[:21]).difference(set(iv_features_1))
len(df_for_model_3.columns.tolist()[:21])
iv_features_1 = ['age_group',
 'balance_category',
 'contact_label_encoded',
 'current_engagement_bins',
 'default_label_encoded',
 'duration_bins',
 'education_label_encoded',
 'housing_label_encoded',
 'is_second_half_of_month',
 'day',             
 'job_label_encoded',
 'loan_label_encoded',
 'marital_label_encoded',
 'month_label_encoded',
 'new_customers',
 'poutcome_label_encoded',
 'prev_engagement_category',
 'pdays_half_yearly_bins',
'cluster_predicted',
'k_means_centroid',
 'y_label_encoded']
len(iv_features_1)

{'ID'}

21

21

In [59]:
iv_features_1 = ['age_group',
 'balance_category',
 'contact_label_encoded',
 'current_engagement_bins',
 'default_label_encoded',
 'duration_bins',
 'education_label_encoded',
 'housing_label_encoded',
 'is_second_half_of_month',
 'day',             
 'job_label_encoded',
 'loan_label_encoded',
 'marital_label_encoded',
 'month_label_encoded',
 'new_customers',
 'poutcome_label_encoded',
 'prev_engagement_category',
 'pdays_half_yearly_bins',
'cluster_predicted',
'k_means_centroid',
 'y_label_encoded']

iv_df_1 = df_for_model_3[iv_features_1]
iv_df_1.head(1)
iv_df_1.shape

,age_group,balance_category,contact_label_encoded,current_engagement_bins,default_label_encoded,duration_bins,education_label_encoded,housing_label_encoded,is_second_half_of_month,day,job_label_encoded,loan_label_encoded,marital_label_encoded,month_label_encoded,new_customers,poutcome_label_encoded,prev_engagement_category,pdays_half_yearly_bins,cluster_predicted,k_means_centroid,y_label_encoded
23354,"[10, 30)","[-8019, 0)",0,"[2, 6)",0,"[120, 300)",1,0,1,23,1,1,2,7,1,3,"[0, 1)","[-1, 0)",3,3,0


(45211, 21)

In [60]:
iv_values_1 = []
for col in iv_df_1.columns:
    if col == 'y_label_encoded': continue
    else:
#         print('WoE and IV for column: {}'.format(col))
        df, iv = calculate_woe_iv(iv_df_1, col, 'y_label_encoded')
        iv_values_1.append(iv.round(2))
#         print(df)
#         print('IV score: {:.2f}'.format(iv))
#         print('\n')
        

In [61]:
list_of_iv_values_1 = sorted(list(zip(iv_features_1, iv_values_1)),key=lambda x: x[1], reverse = True)
iv_features_df_1 = pd.DataFrame(list_of_iv_values_1, columns =['Feature', 'IV_value']) 
iv_features_df_1

,Feature,IV_value
0,duration_bins,1.56
1,poutcome_label_encoded,0.51
2,month_label_encoded,0.44
3,contact_label_encoded,0.30
4,cluster_predicted,0.28
5,pdays_half_yearly_bins,0.24
6,new_customers,0.22
7,prev_engagement_category,0.22
8,housing_label_encoded,0.19
9,age_group,0.18


<font color='navy'> centroids from k-means clustering didn't predict subscription behaviour well. </font> 


In [62]:
top_10_features = [feature[0] for feature in list_of_iv_values_1[:10]]
top_10_features

['duration_bins',
 'poutcome_label_encoded',
 'month_label_encoded',
 'contact_label_encoded',
 'cluster_predicted',
 'pdays_half_yearly_bins',
 'new_customers',
 'prev_engagement_category',
 'housing_label_encoded',
 'age_group']

In [44]:
list_of_features = ['marital_label_encoded', 'loan_label_encoded', 'education_label_encoded', 'current_engagement_bins', 'balance_category', 'pdays_half_yearly_bins']
for feature in list_of_features:
    feature_by_target = df_for_model_3.pivot_table(index = [feature], columns = 'y_label_encoded', values = 'ID', aggfunc = len,margins=True)
    feature_by_target_percen = feature_by_target.div(feature_by_target.iloc[:,-1], axis=0 )*100
    feature_by_target_percen

y_label_encoded,0,1,All
marital_label_encoded,,,
0,88.054542,11.945458,100.0
1,89.876534,10.123466,100.0
2,85.050821,14.949179,100.0
All,88.301520,11.698480,100.0


y_label_encoded,0,1,All
loan_label_encoded,,,
0,87.344273,12.655727,100.0
1,93.318609,6.681391,100.0
All,88.301520,11.698480,100.0


y_label_encoded,0,1,All
education_label_encoded,,,
0,91.373522,8.626478,100.0
1,89.440565,10.559435,100.0
2,84.993610,15.006390,100.0
3,86.429725,13.570275,100.0
All,88.301520,11.698480,100.0


y_label_encoded,0,1,All
current_engagement_bins,,,
"[1, 2)",85.402417,14.597583,100.0
"[2, 6)",89.383150,10.616850,100.0
"[6, 11)",93.478949,6.521051,100.0
"[11, 64)",96.070234,3.929766,100.0
All,88.301520,11.698480,100.0


y_label_encoded,0,1,All
balance_category,,,
"[-8019, 0)",94.423792,5.576208,100.0
"[0, 500)",90.075990,9.924010,100.0
"[500, 3000)",86.277336,13.722664,100.0
"[3000, 102128)",83.668744,16.331256,100.0
All,88.301520,11.698480,100.0


y_label_encoded,0,1,All
pdays_half_yearly_bins,,,
"[-1, 0)",90.842669,9.157331,100.0
"[0, 180)",71.011735,28.988265,100.0
"[180, 872)",80.583856,19.416144,100.0
All,88.301520,11.698480,100.0


<font color='navy'>    1. There seems to be a little variation among marital status and target. remove this feature and re-model  </font> 


In [63]:
train_val_df.pivot_table(index = 'current_engagement_bins', columns = 'y_label_encoded', values = 'ID', aggfunc = len)


y_label_encoded,0,1
current_engagement_bins,,
"[1, 2)",11941,2097
"[2, 6)",16706,1963
"[6, 11)",2348,166
"[11, 64)",909,38
